In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
from functools import reduce
pd.options.mode.chained_assignment = None  # default='warn'
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
import sys
sys.path.append("../../../Functions and Dictionaries/") # Adds higher directory to python modules path
import geodict
namestocommon = geodict.namestocommon
geotogeoid = geodict.geotogeoid
tofullcensus = geodict.tofullcensus
import sqlite3 as sq

In [2]:
conn = sq.connect('../../../Pipeline-Census-Bureau/Outputs/CensusBureau.db')
sql_query = pd.read_sql('SELECT * FROM [SUSB_Annual]', conn)
data = pd.DataFrame(sql_query)
thelist = ['Cheatham', 'Davidson', 'Dickson', 'Houston', 'Humphreys', 'Montgomery', 'Robertson', 'Rutherford', 
          'Stewart', 'Sumner', 'Trousdale', 'Williamson', 'Wilson']
theotherlist = ['Tennessee', 'United states']
data = data.loc[(data['State Name'].isin(theotherlist)) | (data['County'].isin(thelist))]
thefinallist = ['2020', '2021']
data = data.loc[data['Year'].isin(thefinallist)]
data = data.loc[(data['Enterprise Size'] == '2: <20 employees')&(data['NAICS Description'] == 'Total')]

In [3]:
data.head()

,StateFIPS,State Name,CountyFIPS,County,NAICS,NAICS Description,Enterprise Size,Firms,Establishments,Employment,Annual Payroll ($1000),Year,GEO_ID
1,47,Tennessee,001,Anderson,--,Total,2: <20 employees,890,898,3990,174673,2021,0500000US47001
221,47,Tennessee,003,Bedford,--,Total,2: <20 employees,589,590,2336,87002,2021,0500000US47003
393,47,Tennessee,005,Benton,--,Total,2: <20 employees,201,201,875,27110,2021,0500000US47005
500,47,Tennessee,007,Bledsoe,--,Total,2: <20 employees,80,80,292,8755,2021,0500000US47007
551,47,Tennessee,009,Blount,--,Total,2: <20 employees,1566,1570,6949,285919,2021,0500000US47009


In [4]:
conn = sq.connect('../../../Pipeline-Census-Bureau/Outputs/CensusBureau.db')
sql_query = pd.read_sql('SELECT * FROM [SUSB_Annual]', conn)
df = pd.DataFrame(sql_query)
thelist = ['Cheatham', 'Davidson', 'Dickson', 'Houston', 'Humphreys', 'Montgomery', 'Robertson', 'Rutherford', 
          'Stewart', 'Sumner', 'Trousdale', 'Williamson', 'Wilson']
theotherlist = ['Tennessee', 'United States']
df = df.loc[(df['State Name'].isin(theotherlist))&(df['County'].isin(thelist))&
                (df['NAICS Description'] == 'Total')&(df['Enterprise Size'] == '1: Total')]
df = df.loc[(df['Year'] == '2020') | (df['Year'] == '2021')]

In [5]:
df.head()

,StateFIPS,State Name,CountyFIPS,County,NAICS,NAICS Description,Enterprise Size,Firms,Establishments,Employment,Annual Payroll ($1000),Year,GEO_ID
1555,47,Tennessee,021,Cheatham,--,Total,1: Total,600,651,6938,304497,2021,0500000US47021
2555,47,Tennessee,037,Davidson,--,Total,1: Total,16656,21052,445200,29959118,2021,0500000US47037
3170,47,Tennessee,043,Dickson,--,Total,1: Total,970,1047,14631,617007,2021,0500000US47043
6089,47,Tennessee,083,Houston,--,Total,1: Total,98,100,1007,34528,2021,0500000US47083
6135,47,Tennessee,085,Humphreys,--,Total,1: Total,325,342,4400,226298,2021,0500000US47085


In [6]:
data = data[['County', 'Establishments', 'Year']].reset_index(drop = True)
data = data.rename(columns = {'Establishments':'Small Business Establishments'})
df = df[['County', 'Establishments', 'Year']].reset_index(drop = True)
df = df.rename(columns = {'Establishments':'Total Establishments'})

In [7]:
data = data.merge(df, on = ['County', 'Year'])
data.head()

,County,Small Business Establishments,Year,Total Establishments
0,Cheatham,484,2021,651
1,Davidson,12521,2021,21052
2,Dickson,685,2021,1047
3,Houston,73,2021,100
4,Humphreys,237,2021,342


In [8]:
cols = ['Small Business Establishments', 'Total Establishments']
data[cols] = data[cols].astype(float)

In [9]:
data['% of Establishments Small Businesses'] = data['Small Business Establishments']/data['Total Establishments']
thelist = ['2019', '2020']
data = data.loc[data['Year'].isin(thelist)]

In [10]:
data

,County,Small Business Establishments,Year,Total Establishments,% of Establishments Small Businesses
13,Cheatham,470.0,2020,637.0,0.737834
14,Davidson,11905.0,2020,20512.0,0.580392
15,Dickson,658.0,2020,1017.0,0.647001
16,Houston,72.0,2020,100.0,0.720000
17,Humphreys,224.0,2020,329.0,0.680851
18,Montgomery,1870.0,2020,3090.0,0.605178
19,Robertson,804.0,2020,1221.0,0.658477
20,Rutherford,3313.0,2020,5631.0,0.588350
21,Stewart,113.0,2020,164.0,0.689024
22,Sumner,2224.0,2020,3391.0,0.655854


In [11]:
data.to_csv('../data/susb.csv', index = False)